# Working with APIs

API stands for Application programming interface<br>
APIs are a way for applications to talk to eachother<br>
In python we use the requests package to handle the connection<br>
<small><strong>Note: It is requests with an s not request(this is a different package)</strong></small><br>
Most API's communicate with data in the form of JSON<br>
JSON stands for JavaScript Object Notation

### importing requests module

In [2]:
import requests

We will be connecting to the Ergast F1 Racer API today:
http://ergast.com/mrd/

to view JSON data nicely in the Chrome browser install the extension JSONview

In [4]:
# Make sure to attach extension .json at the end of url
url = 'https://ergast.com/api/f1/2008/5/driverStandings.json'

### using the request package to access data from api/url

In [10]:
response = requests.get(url)

### checking and making sure we got a successful response from our API
#### status codes: 200 = ok/successful, 400 = bad/unsuccessful

In [11]:
print(response.ok)

True


### view the response as a JSON with .json()

In [14]:
data = response.json()
print(data)

MRData


### Get the Driver Standings Data from JSON

In [41]:
driverStandingsData = data['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']
print(driverStandingsData)

[{'position': '1', 'positionText': '1', 'points': '35', 'wins': '2', 'Driver': {'driverId': 'raikkonen', 'permanentNumber': '7', 'code': 'RAI', 'url': 'http://en.wikipedia.org/wiki/Kimi_R%C3%A4ikk%C3%B6nen', 'givenName': 'Kimi', 'familyName': 'Räikkönen', 'dateOfBirth': '1979-10-17', 'nationality': 'Finnish'}, 'Constructors': [{'constructorId': 'ferrari', 'url': 'http://en.wikipedia.org/wiki/Scuderia_Ferrari', 'name': 'Ferrari', 'nationality': 'Italian'}]}, {'position': '2', 'positionText': '2', 'points': '28', 'wins': '2', 'Driver': {'driverId': 'massa', 'permanentNumber': '19', 'code': 'MAS', 'url': 'http://en.wikipedia.org/wiki/Felipe_Massa', 'givenName': 'Felipe', 'familyName': 'Massa', 'dateOfBirth': '1981-04-25', 'nationality': 'Brazilian'}, 'Constructors': [{'constructorId': 'ferrari', 'url': 'http://en.wikipedia.org/wiki/Scuderia_Ferrari', 'name': 'Ferrari', 'nationality': 'Italian'}]}, {'position': '3', 'positionText': '3', 'points': '28', 'wins': '1', 'Driver': {'driverId': '

### Use a for loop to display all driver full names

In [24]:
for driver in driverStandingsData:
    print(f"{driver['Driver']['givenName']} {driver['Driver']['familyName']}")

Kimi Räikkönen
Felipe Massa
Lewis Hamilton
Robert Kubica
Nick Heidfeld
Heikki Kovalainen
Mark Webber
Fernando Alonso
Jarno Trulli
Nico Rosberg
Kazuki Nakajima
Jenson Button
Sébastien Bourdais
David Coulthard
Timo Glock
Giancarlo Fisichella
Rubens Barrichello
Nelson Piquet Jr.
Takuma Sato
Anthony Davidson
Adrian Sutil
Sebastian Vettel


### Create a function getting certain driver info (givenName, familyName, dateOfBirth, wins, team)

In [59]:
def returnDriverFromPosition(position):
    for driver in driverStandingsData:
        if driver['position'] == str(position):
            driverConstructor = driver['Constructors'][0]
            driverInfo = driver['Driver']

            return {'name': f"{driverInfo['givenName']} {driverInfo['familyName']}",
                    'dateOfBirth': driverInfo['dateOfBirth'],
                    'wins': driver['wins'],
                    'team': driverConstructor['name']}

print(returnDriverFromPosition(1))

{'name': 'Kimi Räikkönen', 'dateOfBirth': '1979-10-17', 'wins': '2', 'team': 'Ferrari'}


### Making a more flexible function to handle different years and rounds

In [63]:
def returnDriver(year, raceRound, position):
    url = f'https://ergast.com/api/f1/{year}/{raceRound}/driverStandings.json'
    response = requests.get(url)

    if not response.ok:
        return response.status_code

    data = response.json()

    driverStandingsData = data['MRData']['StandingsTable']['StandingsLists'][0]['DriverStandings']

    for driver in driverStandingsData:
        if driver['position'] == str(position):
            driverConstructor = driver['Constructors'][0]
            driverInfo = driver['Driver']

            driverDict = {'name': f"{driverInfo['givenName']} {driverInfo['familyName']}",
                    'dateOfBirth': driverInfo['dateOfBirth'],
                    'wins': driver['wins'],
                    'team': driverConstructor['name']}

            return driverDict

print(returnDriver(2012, 4, 1))
    

{'name': 'Sebastian Vettel', 'dateOfBirth': '1987-07-03', 'wins': '1', 'team': 'Red Bull'}


# <strong>Homework</strong>
check out this Pokemon API https://pokeapi.co/
Use the requests package to connect to this API and get and store data for 5 different pokemon.
Get the pokemons: name, atleast one ability's name, base_experience, and the URL for its sprite (an image that shows up on screen) for the 'front_shiny', attack base_stat, hp base_stat, defense base_stat

In [2]:
import requests
from ui import clearCurrentCellOutput
from pokedex import pokedex

def displayPokemon(id):
    def screenLine(label, data):
        def addSpaces():
            spaces = ""
            for space in range(0, pokedex['screenWidth'] - (4 + len(label) + len(data))):
                spaces += " "

            return spaces

        return f"{pokedex['screenStart']} {label} {data}{addSpaces()}{pokedex['screenEnd']}"
    
    def displayPokedex():
        pokedexStr = ""
        pokedexStr += pokedex['top'] + '\n'
        for label, data in pokemonInfoDict.items():
            pokedexStr += (screenLine(label, str(data).title())) + '\n'
        pokedexStr += pokedex['bot']

        return pokedexStr

    url = f"https://pokeapi.co/api/v2/pokemon/{id}"
    response = requests.get(url)

    if not response.ok:
        return response.status_code
    else:
        data = response.json()

        name = data['name']
        abilities = [ability['ability']['name'] for ability in data['abilities'] if ability['is_hidden'] == False]
        try:
            hiddenAbility = [ability['ability']['name'] for ability in data['abilities'] if ability['is_hidden'] == True][0]
        except:
            hiddenAbility = 'None'
        baseExp = data['base_experience']
        spriteURL = data['sprites']['front_default']
        spriteShinyURL = data['sprites']['front_shiny']
        baseStats = {stat['stat']['name'].upper() + ':': stat['base_stat'] for stat in data['stats']}
        pokemonType = [pkmnType['type']['name'] for pkmnType in data['types']]
        pokedexID = data['id']

        pokemonType = f"{pokemonType[0]}{'/' + pokemonType[1] if len(pokemonType) > 1 else ''}"

        labels = ["Name:", "ID:", "Type:", "Ability 1:", "Ability 2:", "Hidden:", "Base Exp:"] + list(baseStats.keys())
        pkmnInfo = [name, pokedexID, pokemonType, abilities[0], abilities[1] if len(abilities) > 1 else 'None', hiddenAbility, baseExp] + list(baseStats.values())

        pokemonInfoDict = dict(zip(labels, pkmnInfo))

        return displayPokedex()

while True:
    currentInput = input("Please enter a Pokemon (name or pokedex #) or 0 (zero) to exit: ")

    if currentInput == '0':
        break
    else:
        clearCurrentCellOutput()
        try:
            print(displayPokemon(currentInput))
        except IndexError:
            print("Pokedex # out of bounds...")
    

############################################                                    
###.#,,//#%/%# &@,&#########################                                    
## *****,,%##########%@%&%###################%%%%%%%#%%                         
##/ *****@########%&&######################%%#########&%&#%                     
#######%#####%&%%&%#####################%%&%#############%&&%#                  
###########################################################%#&&&@%#%############
#(*.........,,.,.,,,,,,,,,,,,,,,,,,,**########################################@&
#(* Name: Mew                        *#########******************************#&&
#(* ID: 151                          *#########******************************#&&
#(* Type: Psychic                    *#########******************************#&&
#(* Ability 1: Synchronize           *#########******************************#&&
#(* Ability 2: None                  *#########******************************#&&
#(* Hidden: None            